# Lecture 22 - Large Language Models (Part 2)

[![View notebook on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/avakanski/Fall-2025-Applied-Data-Science-with-Python/blob/main/docs/Lectures/Theme_3-Model_Engineering/Lecture_22-LLMs_Part_2/Lecture_22-LLMs_Part_2.ipynb)
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/avakanski/Fall-2025-Applied-Data-Science-with-Python/blob/main/docs/Lectures/Theme_3-Model_Engineering/Lecture_22-LLMs_Part_2/Lecture_22-LLMs_Part_2.ipynb)

<a id='top'></a>

- [22.2 Vision-Language Models](#22.2-vision-language-models)
  - [22.2.1 VLM Architectures](#22.2.1-vlm-architectures)
  - [22.2.2 Benchmarking VLMs](#22.2.2-benchmarking-vlms)
  - [22.2.3 VLMs Importance](#22.2.3-vlms-importance)
  - [22.2.4 VLM Finetuning](#22.2.4-vlm-finetuning)

## 22.2 Vision-Language Models <a name='22.2-vision-language-models'></a>

**Vision Language Models (VLMs)** are multimodal systems that jointly process and reason over visual (images, videos) and linguistic (text) information. By integrating the two modalities, VLMs can understand and communicate about visual content using natural language.

VLMs take both an image and its textual description as input and generate text as output. Building datasets for such models requires large scale collection of images and corresponding text, typically in the form of image captions or descriptive phrases. Several very large such datasets exist that have been essential for training modern VLMs and contain millions or billions of image-text pairs, with descriptions in English and other languages. For instance, the [LAION-5B](https://laion.ai/blog/laion-5b/) dataset has 5.8 billion image-text examples, and the [PMD (Public Model Dataset)](https://huggingface.co/datasets/facebook/pmd) contains 70 million image-text examples.

<img src="images/vlm_structure.png" width="450">

*Figure: VLM structure.* Source: [1].

During training, a VLM learns to map visual and textual representations into a joint embedding space. This mapping allows the model to associate visual features (shapes, colors, spatial relations) with linguistic concepts, enabling it to generalize to a wide range of vision tasks and perform zero-shot inference on unseen examples.

The example below illustrates a VLM performing tasks such as object localization, segmentation, visual question answering, and image learning with instructions. The user prompts are shown on the left, and the model responses are given on the right. As demonstrated in this example, VLM models can not only interpret the semantic content of images, but can also understand spatial relations in images, such as identifying relative positions of objects, or generating segmentation masks. VLMs can also output bounding boxes of objects, and perform other spatial tasks.

<img src="images/VLM_capabilities.jpg" width="600">

*Figure: VLM prompts and responses.* Source: [2].

In general, VLMs can perform various multimodal tasks including:

- Image and video captioning/summarization: generate context-aware descriptions of images or video frames.
- Visual question answering (VQA): answer open-ended questions based  on visual content.
- Image-based reasoning: provide explanations or logical reasoning about visual scenes.
- Multimodal dialogues: engage in conversations involving visual inputs.
- Text-to-image search: retrieve images, figures, or diagrams in documents that match a textual query.
- Image generation: generate new images based on textual prompts.

### 22.2.1 VLM Architectures<a name='22.2.1-vlm-architectures'></a>

####  **VLMs with Aligned Multimodal Embeddings**

Numerous architectures of VLMs have been proposed in recent years that employ different strategies to integrate visual and textual information.

One common VLM workflow is shown in the above figure titled VLM Structure, and includes the following main components.

**Multimodal inputs**. Input modalities in VLMs include *visual inputs* (images, PDF documents, or videos) and *textual inputs* (captions, question-answer pairs, or instructions).

**Encoding Modalities.** A *vision encoder* transforms the visual input into numerical representations, referred to as visual embeddings. The vision encoder in VLMs is commonly a variant of the Vision Transformer (ViT) architecture. A *text encoder* converts textual prompts into text embeddings. The text encoder is typically a Transformer-based encoder often pretrained on large text corpora.

**Projection into a multimodal embedding space.** The visual and textual embeddings are next projected into a shared multimodal embedding space. This step is achieved through a *projector head*, which is usually implemented as a small Transformer block or a block of fully-connected layers. The projector layers align the visual and textual respresentations into a shared embedding space, allowing the model to reason simultaneously across images and language.

The shared embedding space enables VLMs to link textual concepts (e.g., cat) with corresponding visual evidence (a cat's color or location in the image), allowing reasoning across both language and vision. For instance, the model understands "cat" not just as a word, but also as a visual object in the image.

One representative model of this type of VLM architectures is CLIP (Contrastive Language-Image Pretraining). It is one of the earliest models that introduced vision-language alignment through contrastive learning. CLIP employs both a vision encoder and a text encoder, and learns a shared embedding space in which images and their textual descriptions are semantically aligned. *Contrastive learning* in CLIP is employed to associate visual and textual content by maximizing the similarity between matched image and text pairs and minimizing the similarity between mismatched ones. In the figure below, the image encoder outputs image embeddings $I_1, I_2, I_3, ..., I_N$, and the text encoder outputs text embeddings $T_1, T_2, T_3, ..., T_N$. The model computes a similarity score between each pair of image embeddings (e.g., $I_i$) and text embeddings (e.g., $T_j$) to align them into a shared embedding space $I_i\cdot T_j$. Similarity scores are calculated using the dot product (i.e., cosine similarity) $I_i\cdot T_j$ between image and text embeddings. As training progresses, the two encoders are updated so that image and text representations corresponding to similar concepts are drawn close to each other in the shared embedding space, while dissimilar concepts are pushed apart.

<img src="images/CLIP.png" width="550">

*Figure: CLIP architecture.* Source: [5].

The pretrained vision encoder of CLIP has been widely adopted as a vision encoder component in numerous later VLM variants.

#### **VLMs with Fused Multimodal Embeddings**

Other VLM architectures fuse together image and text representations into joint multimodal embeddings to enable more advanced visual reasoning. These models typically employ a *text decoder* network from pretrained LLM, which generates a text output.

The following figure illustrates the workflow of this class of VLMs. The visual embeddings from the vision encoder are first matched by the multimodal projector to the corresponding text embeddings of the LLM. The LLM decoder receives a combined representation consisting of visual embeddings (image tokens) from the projector and text embeddings (text tokens) from the user's prompt or question. The LLM decoder serves as the text generation backbone, and using the combined image and text tokens, it generates a textual output in an autoregressive manner, one token at a time. Each new token is conditioned on previously generated tokens and on the fused multimodal embeddings.

<img src="images/vlm_fused_embeddings.png" width="400">

*Figure: Fused multimodal embeddings.* Source: [2].

Training VLMs with fused vision and text embeddings typically involves multiple stages, as shown in the next figure. In the first stage only the multimodal projector is trained while keeping the image encoder and the LLM text decoder frozen. This is followed by a second stage that involves additional finetuning of the multimodal projector and parts of the text decoder, while keeping the image encoder frozen.

<img src="images/vlm_training.png" width="550">

*Figure: VLM training.* Source: [2].

In addition, in some VLM architectures, the vision encoder is also finetuned in the second stage to further improve cross-modal reasoning.

Representative models of this VLM architectures are BLIP and Flamingo, which introduced cross-attention mechanisms (similar to the cross-attention module connecting the encoder and decoder in standard Transformer networks). These mechanisms enable direct fusion of image and text embeddings into a single multimodal representation, allowing the models to reason more efficiently over both modalities.

These models laid the foundations for the modern multimodal foundation models, such as Gemini 2.5 (Google), GPT-5 (OpenAI), Claude Opus 4 (Anthropic), Qwen-VL (Alibaba), and Mistral 3.1 (Mistral AI). These models represent advanced VLMs, and typically employ fused image and text representations within a unified embedding space, to provide advanced  visual comprehension, reasoning, and dialog capabilities.

Also, many open-source VLM alternatives have made this fused-architecture functionality widely accessible to the research community, and include LLaVA, Qwen-VL, LLaMA 3.2 Vision, InternVL, Pixtral, and others.


### 22.2.2 Benchmarking VLMs<a name='22.2.2-benchmarking-vlms'></a>

Performance of VLMs is assessed using multimodal benchmarks that evaluate models on a variety of tasks, such as reasoning, visual question answering, document comprehension, video understanding, and other tasks. Most benchmarks consist of a set of images with associated questions, often posed as multiple-choice questions. Popular benchmarks are [MMMU](https://mmmu-benchmark.github.io/), [Video-MME](https://video-mme.github.io/home_page.html), [MathVista](https://mathvista.github.io/), and [ChartQA](https://github.com/vis-nlp/ChartQA). MMMU is the most comprehensive benchmark, and contains 11.5K multimodal challenges that require knowledge and reasoning across different disciplines such as arts and engineering.

Several VLM-specific leaderboards provide comparative rankings across diverse metrics. [Vision Arena](https://lmarena.ai/leaderboard/vision) ranks models based on anonymous voting of model outputs by human preferences. [Open VLM Leaderboard](https://huggingface.co/spaces/opencompass/open_vlm_leaderboard) provides comparative ranking of VLMs according to different metrics and average scores.

### 22.2.3 VLMs Importance<a name='22.2.3-vlms-importance'></a>

Traditional computer vision (CV) models are constrained to learning from a predefined and fixed set of categories or objects for image classification or object detection (e.g., identify whether an image contains a cat or a dog). Moreover, these tasks require users to manually label a large number of images with a specific category for classification, or assign bounding boxes to multiple objects in each image for object detection, which is a tedious, time-consuming, and expensive process.

Conversely, VLMs are trained with more  detailed textual descriptions of images, where for example an image can contain cats, dogs, and other objects, as well as the text description can provide additional contextual information (e.g., the cat is sitting, the dog is running, etc.). Learning from rich natural language descriptions allows VLMs to better understand visual scenes without limiting the learning to a narrow set of visual concepts comprising a fixed number of classes and objects. Also, it eliminates the need for exhaustive manual image labeling and extends VLMs' utility beyond traditional CV tasks like classification or detection to new capabilities including reasoning, summarization, question answering, and interactive dialogue, by simply changing the text prompt.

VLMs have been applied across many domains and industries, and offer great potential to enhance visual perception. For instance, they can be used to review videos and extract insights for industrial inspection and robotics (detect faults, monitor operations, identify anomalies in real time), safety and infrastructure monitoring (recognize floods, fires, or traffic hazards), retail and logistics (track empty shelves, detect misplaced items, identify supply-chain bottlenecks), and numerous other tasks.

### 22.2.4 VLM Finetuning <a name='22.2.4-vlm-finetuning'></a>

We are excited to announce that TRL’s SFTTrainer now includes experimental support for Vision Language Models! We provide an example here of how to perform SFT on a Llava 1.5 VLM using the llava-instruct dataset which contains 260k image-conversation pairs. The dataset contains user-assistant interactions formatted as a sequence of messages. For example, each conversation is paired with an image that the user asks questions about.

## References <a name='references'></a>

1. Vision Language Models (VLMs) Explained - GeeksForGeeks, available at [https://www.geeksforgeeks.org/artificial-intelligence/vision-language-models-vlms-explained/](https://www.geeksforgeeks.org/artificial-intelligence/vision-language-models-vlms-explained/).
2. Vision Language Models Explained - Hugging Face Blog, by Merve,
Edward Beeching, available at [https://huggingface.co/blog/vlms](https://huggingface.co/blog/vlms).
3. Understanding Vision-Language Models (VLMs): A Practical Guide, by Pietro Bolcato, available at [https://medium.com/@pietrobolcato/understanding-vision-language-models-vlms-a-practical-guide-8da18e9f0e0c](https://medium.com/@pietrobolcato/understanding-vision-language-models-vlms-a-practical-guide-8da18e9f0e0c).
4. What Are Vision Language Models, by NVIDIA, available at [https://www.nvidia.com/en-us/glossary/vision-language-models/](https://www.nvidia.com/en-us/glossary/vision-language-models/).
5. CLIP: Connecting text and images, OpenAI, available at [https://openai.com/index/clip/](https://openai.com/index/clip/).